# Italian Exercise Generator - Colab Inference API

This notebook creates a FastAPI inference service for the Italian Exercise Generator model.

**What it does:**
- Loads your GRPO-trained `TeacherPet_italian_grpo` model with vLLM (4.4x faster inference)
- Exposes a FastAPI endpoint for generating Italian exercises
- Creates a public tunnel via ngrok so your local API can access it

**Model: TeacherPet_italian_grpo**
- Training: GRPO (Group Relative Policy Optimization) - reinforcement learning
- Advantages: Better grammar accuracy, tense consistency, reduced hallucination
- No LoRA merging needed - this is a full fine-tuned model

**Usage:**
1. Run all cells in order
2. Copy the ngrok URL from the output
3. Export it locally: `export INFERENCE_API_URL="https://your-url.ngrok.io"`
4. Start your local API: `./run_api.sh`
5. Your local API will now use Colab GPU for homework generation!

In [1]:
# Cell 1: Install dependencies
!pip install fastapi uvicorn pyngrok vllm nest-asyncio spacy -q
!python -m spacy download it_core_news_sm
print("✅ Dependencies installed")
print("✅ Italian NLP model installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0

In [2]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted")

Mounted at /content/drive
✅ Google Drive mounted


In [3]:
# Cell 3: Setup paths and verify model exists
import os
import sys

PROJECT_ROOT = "/content/drive/MyDrive/Colab Notebooks/italian_teacher"
MODEL_PATH = os.path.join(PROJECT_ROOT, "models/TeacherPet_italian_grpo")

# Add project to Python path for imports
sys.path.insert(0, PROJECT_ROOT)

# Verify model exists
if not os.path.exists(MODEL_PATH):
    print(f"❌ Model not found at: {MODEL_PATH}")
    print("Please update MODEL_PATH to point to your TeacherPet_italian_grpo model")
    print("\nExpected structure:")
    print("  models/TeacherPet_italian_grpo/")
    print("    ├── config.json")
    print("    ├── tokenizer.json")
    print("    ├── model-*.safetensors")
    print("    └── ...")
else:
    print(f"✅ GRPO model found at: {MODEL_PATH}")
    print(f"✅ Model: TeacherPet_italian_grpo")
    print(f"✅ Training: GRPO (Group Relative Policy Optimization)")
    print(f"✅ Project root: {PROJECT_ROOT}")

✅ GRPO model found at: /content/drive/MyDrive/Colab Notebooks/italian_teacher/models/TeacherPet_italian_grpo
✅ Model: TeacherPet_italian_grpo
✅ Training: GRPO (Group Relative Policy Optimization)
✅ Project root: /content/drive/MyDrive/Colab Notebooks/italian_teacher


In [4]:
# Cell 4: Verify model files (GRPO model is already a full model, no merging needed)
import torch
import os

print("✅ TeacherPet_italian_grpo is a full GRPO-trained model")
print("✅ No LoRA merging required - model is ready to use with vLLM")
print("")

# Check for key model files
model_files = [
    "config.json",
    "tokenizer.json",
    "tokenizer_config.json"
]

print("📋 Checking model files:")
all_files_exist = True
for file in model_files:
    file_path = os.path.join(MODEL_PATH, file)
    exists = os.path.exists(file_path)
    status = "✅" if exists else "❌"
    print(f"   {status} {file}")
    if not exists:
        all_files_exist = False

# Check for safetensors files
import glob
safetensors_files = glob.glob(os.path.join(MODEL_PATH, "*.safetensors"))
if safetensors_files:
    print(f"   ✅ Found {len(safetensors_files)} model weight file(s)")
else:
    print(f"   ❌ No .safetensors files found")
    all_files_exist = False

if all_files_exist:
    print(f"\n✅ Ready to load model for vLLM: {MODEL_PATH}")
else:
    print(f"\n❌ Some model files are missing. Please check your model directory.")

✅ TeacherPet_italian_grpo is a full GRPO-trained model
✅ No LoRA merging required - model is ready to use with vLLM

📋 Checking model files:
   ✅ config.json
   ✅ tokenizer.json
   ✅ tokenizer_config.json
   ✅ Found 4 model weight file(s)

✅ Ready to load model for vLLM: /content/drive/MyDrive/Colab Notebooks/italian_teacher/models/TeacherPet_italian_grpo


In [5]:
# Cell 5: Load GRPO model with vLLM (~30 seconds)
from vllm import LLM

print("⏳ Loading TeacherPet_italian_grpo with vLLM for fast inference...")
print("📊 Model: GRPO-trained (reinforcement learning optimized)")
print("")

llm = LLM(
    model=MODEL_PATH,
    tensor_parallel_size=1,
    dtype="half",
    max_model_len=2048,
    gpu_memory_utilization=0.85,
    trust_remote_code=True
)

print("\n✅ TeacherPet_italian_grpo model loaded with vLLM!")
print(f"🔥 GPU: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")
print(f"💾 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f}GB")
print(f"🎯 Training: GRPO (Group Relative Policy Optimization)")
print(f"✨ Benefits: Better grammar, tense consistency, reduced hallucination")

INFO 11-03 11:57:35 [__init__.py:216] Automatically detected platform cuda.
⏳ Loading TeacherPet_italian_grpo with vLLM for fast inference...
📊 Model: GRPO-trained (reinforcement learning optimized)

INFO 11-03 11:57:47 [utils.py:233] non-default args: {'trust_remote_code': True, 'dtype': 'half', 'max_model_len': 2048, 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'model': '/content/drive/MyDrive/Colab Notebooks/italian_teacher/models/TeacherPet_italian_grpo'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 11-03 11:58:07 [model.py:547] Resolved architecture: LlamaForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 11-03 11:58:07 [model.py:1733] Casting torch.bfloat16 to torch.float16.
INFO 11-03 11:58:07 [model.py:1510] Using max model len 2048
INFO 11-03 11:58:10 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 11-03 11:58:19 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-03 12:03:48 [llm.py:306] Supported_tasks: ['generate']

✅ TeacherPet_italian_grpo model loaded with vLLM!
🔥 GPU: NVIDIA L4
💾 GPU Memory: 0.00GB
🎯 Training: GRPO (Group Relative Policy Optimization)
✨ Benefits: Better grammar, tense consistency, reduced hallucination


In [6]:
# Cell 6: Create FastAPI application
import nest_asyncio
from src.api.inference import create_inference_app

# Allow nested event loops (required for Colab)
nest_asyncio.apply()

# Port for Colab API (8001 to avoid conflict with local API on 8000)
COLAB_PORT = 8001

# Create the FastAPI app
app = create_inference_app(llm, port=COLAB_PORT)

print(f"✅ FastAPI application created (port {COLAB_PORT})")
print(f"📋 Version: 3.0.0 (GRPO)")
print(f"🎯 Model: TeacherPet_italian_grpo")
print("🚀 Ready to start server!")

✅ FastAPI application created (port 8001)
📋 Version: 3.0.0 (GRPO)
🎯 Model: TeacherPet_italian_grpo
🚀 Ready to start server!


In [7]:
# Cell 7: Setup ngrok tunnel
from pyngrok import ngrok

# Set your ngrok auth token (get free token at https://ngrok.com)
NGROK_AUTH_TOKEN = "33VKJ1gR2EjYu8WvlRmSqOiUiJk_2qCL7X8Kp4vHTUAcu4xvh"

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Create tunnel
tunnel = ngrok.connect(COLAB_PORT)
public_url = str(tunnel.public_url)

print("🌐 ngrok tunnel created!")
print(f"\n📍 Public URL: {public_url}")
print(f"\n🔗 API Endpoints:")
print(f"   Health: {public_url}/health")
print(f"   Generate: {public_url}/generate")
print(f"\n✅ Copy the public URL above for use in your local environment")

🌐 ngrok tunnel created!

📍 Public URL: https://orthoscopic-nonengrossingly-lashon.ngrok-free.dev

🔗 API Endpoints:
   Health: https://orthoscopic-nonengrossingly-lashon.ngrok-free.dev/health
   Generate: https://orthoscopic-nonengrossingly-lashon.ngrok-free.dev/generate

✅ Copy the public URL above for use in your local environment


In [8]:
# Cell 8: Start FastAPI server
import uvicorn
from threading import Thread
import time
import requests
import json

print(f"🚀 Starting FastAPI server on port {COLAB_PORT}...")

# Create uvicorn config
config = uvicorn.Config(
    app=app,
    host="0.0.0.0",
    port=COLAB_PORT,
    log_level="error"
)

# Create server
server = uvicorn.Server(config)

# Start in background thread
def run_server():
    import asyncio
    asyncio.run(server.serve())

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to be ready
print("⏳ Waiting for server to start...")
time.sleep(3)

# Test if it's working
try:
    response = requests.get(f"http://localhost:{COLAB_PORT}/health", timeout=2)

    if response.status_code == 200:
        print("\n✅ SERVER IS RUNNING!")
        print(f"📡 Listening on http://0.0.0.0:{COLAB_PORT}\n")

        print("🧪 Health check response:")
        print(json.dumps(response.json(), indent=2))

        print("\n" + "="*70)
        print("🌐 YOUR NGROK PUBLIC URL:")
        print("="*70)
        print(f"\n{public_url}\n")
        print("="*70)

        print("\n📋 COPY AND RUN ON YOUR MAC:\n")
        print(f'export INFERENCE_API_URL="{public_url}"')
        print("./run_api.sh")

        print("\n" + "="*70)
        print("\n⚡ Server is running! Keep this notebook open!")
        print("🛑 To stop: Runtime → Interrupt execution")
        print("="*70)

except Exception as e:
    print(f"\n❌ Server failed to start: {e}")
    print("\n🔄 Try this:")
    print("   1. Runtime → Restart runtime")
    print("   2. Re-run all cells")

🚀 Starting FastAPI server on port 8001...
⏳ Waiting for server to start...

✅ SERVER IS RUNNING!
📡 Listening on http://0.0.0.0:8001

🧪 Health check response:
{
  "status": "healthy",
  "gpu_available": true,
  "gpu_memory_allocated_gb": 0.0,
  "model_loaded": true,
  "port": 8001
}

🌐 YOUR NGROK PUBLIC URL:

https://orthoscopic-nonengrossingly-lashon.ngrok-free.dev


📋 COPY AND RUN ON YOUR MAC:

export INFERENCE_API_URL="https://orthoscopic-nonengrossingly-lashon.ngrok-free.dev"
./run_api.sh


⚡ Server is running! Keep this notebook open!
🛑 To stop: Runtime → Interrupt execution
